In [1]:
import os
import pandas as pd

df_path = "data/250528_PCG_assessment_report_Q1-2025_controlperrow.xlsx"
ori_df = pd.read_excel(df_path)
ori_df["company"] = "PCG"
ori_df["process_desc"] = ""
ori_df["rootcause_desc"] = ""

df = ori_df.copy()
print(df.shape)
df.head()

(175, 25)


,No.,User,BU Name,Risk Category,Risk SubCategory,Risk Item,Risk Description,Root Cause,Process,likelihood_combined,...,Control,Control Description,EC-Root Cause,EC-Process,Design Score,Effective Score,Control Owner,company,process_desc,rootcause_desc
0,1,nangnoi.nhu,PCG_Dry food manufacturing,Operational Risk,Operation,Supply shortage,การจัดหาวัตถุดิบไม่ทันตามกระบวนการผลิต อาจทำให...,Supplier / Vendor shortage: สินค้าบางประเภทมี ...,Marketing and Sales: กรณีวัตถุดิบไม่เพียงพอต้อ...,3,...,NaN,NaN,NaN,NaN,0,0,NaN,PCG,,
1,2,piraya.amo,PCG_Marketing,Strategic Risk,Competitor,Intense market competition,ความเสี่ยงจากการแข่งขันในตลาดบนช่องทางออนไลน์แ...,High number of competitors in the market: การแ...,Planning: ต้องใช้ข้อมูลด้านการตลาดและพฤติกรรมข...,4,...,NaN,NaN,NaN,NaN,0,0,NaN,PCG,,
2,3,piraya.amo,PCG_Marketing,Strategic Risk,Social,Negative media coverage,หากมีการรายงานข่าวเชิงลบ / รีวิวเชิงลบใน Socia...,Other: หากมีการรีวิวและคอมเมนต์เชิงลบ ในช่องทา...,Marketing and Sales: ได้รับผลโดยตรงหากแบรนด์ถู...,4,...,NaN,NaN,NaN,NaN,0,0,NaN,PCG,,
3,4,pongsak.lio,PCG_QA&QC,Operational Risk,Operation,Poor supply quality,ความไม่สม่ำเสมอของคุณภาพ: วัตถุดิบอาจมีความไม่...,Inadequate quality control processes: การวิเคร...,"Quality Assurance: -, Other: -",3,...,มาตรการตรวจสอบคุณภาพ,มี Procedure การตรวจและการทดสอบ มี Work Instru...,Inadequate quality control processes: การวิเคร...,"Quality Assurance: -, Other: -",3,4,ปองพล ไลออน(ควบคุมคุณภาพ),PCG,,
4,5,pongsak.lio,PCG_QA&QC,Operational Risk,Operation,Operational inefficiency,ระบบการจัดการควบคุมคุณภาพล้มเหลว เช่น แผนการสุ...,"Lack of accountability: -, Insufficient traini...",Other: -,2,...,ระบบการจัดการกระบวนการควบคุมคุณภาพ,มี Procedure การตรวจและการทดสอบ มี Work Instru...,"Lack of accountability: -, Insufficient traini...",Other: -,3,4,ปองพล ไลออน (ควบคุมคุณภาพ),PCG,,


In [2]:
selected_risk_cat = [
    "Operational Risk",
    "Strategic Risk",
    "Credit Risk",
    "Market Risk",
    "Liquidity Risk",
]

In [3]:
file_name = os.path.basename(df_path)
date_stamp = file_name.split("_")[0]
date_stamp

'250528'

In [4]:
# calc risk score
# Select columns that start with 'financial' but do not end with '_note'
# riskscore_columns = df.loc[:, (df.columns.str.startswith('Financial') | df.columns.str.startswith('NonFinancial') | df.columns.str.startswith('Likelihood')) & ~df.columns.str.endswith('_notes') & ~df.columns.str.endswith('_justification')]
df = ori_df.copy()


df["risk_score"] = df["likelihood_combined"] * df["impact_combined"]


# Define the function to determine the risk level
def determine_risk_level(row, type):
    if type == "fin":
        riskscore = "risk_score_fin"
        riskimpact = "impact_fin_combined"
    elif type == "nonfin":
        riskscore = "risk_score_nonfin"
        riskimpact = "impact_nonfin_combined"
    else:
        riskscore = "risk_score"
        riskimpact = "impact_combined"
    if row[riskscore] >= 20:
        return 4
    elif 10 <= row[riskscore] <= 16:
        return 3
    elif 4 <= row[riskscore] <= 9:
        return 2
    elif (0 < row[riskscore] < 4) or (row[riskscore] == 4 and row[riskimpact] == 2):
        return 1
    elif row[riskscore] == 0:
        return 0
    # else:
    #     return 'Undefined'


# Apply the function to each row in the DataFrame
df["risk_level"] = df.apply(determine_risk_level, type="combined", axis=1)

In [5]:
df["risk_level"].value_counts()

risk_level
2    114
1     59
4      1
3      1
Name: count, dtype: int64

In [6]:
df.rename(
    columns={
        "Risk Category": "risk_cat",
        "Risk Item": "risk",
        "Risk Description": "risk_desc",
        "Root Cause": "rootcause",
        "Process": "process",
        "Control": "control_name",
        "Control Description": "control_des",
        "EC-Root Cause": "control_rootcause",
        "EC-Process": "control_process",
    },
    inplace=True,
)

In [7]:
debug_df = df[df["company"] == "PCG"]
debug_df = debug_df[debug_df["risk"].str.lower() == "poor supply quality"]
debug_df[["risk", "risk_score", "likelihood_combined", "impact_combined"]]

,risk,risk_score,likelihood_combined,impact_combined
3,Poor supply quality,6,3,2
43,Poor supply quality,3,1,3
108,Poor supply quality,3,3,1
171,Poor supply quality,6,3,2


In [8]:
df2 = df.groupby(["company", "risk"], as_index=False).agg(
    {
        "risk_desc": set,
        "risk_cat": "first",
        # "iscustom": set,  #'first', # risk
        # "company_sector": set,
        # "risk_subcat": set,
        # "control_name": set,
        # "control_des": set,
        "rootcause": set,
        "rootcause_desc": set,
        "process": set,
        "process_desc": set,
        "risk_level": max,
        "likelihood_combined": max,
        "impact_combined": max,
        # "impact_fin_combined": max,
        # "impact_nonfin_combined": max,
    }
)
# TODO: check if other df# are recalculating this since max(impact)*max(likelihood) != max(risk_score)
df2["risk_score"] = df2["impact_combined"] * df2["likelihood_combined"]
# df2["risk_score_fin"] = df2["impact_fin_combined"] * df2["likelihood_combined"]
# df2["risk_score_nonfin"] = df2["impact_nonfin_combined"] * df2["likelihood_combined"]

df2["risk_level"] = df2.apply(determine_risk_level, type="combine", axis=1)
# df2["risk_level_fin"] = df2.apply(determine_risk_level, type="fin", axis=1)
# df2["risk_level_nonfin"] = df2.apply(determine_risk_level, type="nonfin", axis=1)

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_75694/2672502393.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df2 = df.groupby(["company", "risk"], as_index=False).agg(


In [9]:
print(df2.shape)
df2.head()

(66, 12)


,company,risk,risk_desc,risk_cat,rootcause,rootcause_desc,process,process_desc,risk_level,likelihood_combined,impact_combined,risk_score
0,PCG,Accounting errors,{การแจ้งรายละเอียดค่าใช้จ่ายที่เกิดขึ้นในคลังม...,Operational Risk,"{Mistakes in calculations, data entry, or judg...",{},{Finance and Accounting: -},{},1,2,1,2
1,PCG,Business interruption from fire hazards,{กระบวนการทำงานของบริษัทมีความเสี่ยงที่อาจก่อใ...,Operational Risk,"{Inadequate Fire Safety Training: -, Insuffici...",{},"{Maintenance: -, Engineering: -, Production: -...",{},2,2,3,6
2,PCG,Business interruption from flood,{น้ำท่วมพื้นที่คลังสินค้า หรือบริเวณใกล้เคียง},Operational Risk,"{Inadequate Emergency Training: -, Poor Commun...",{},{Distribution Center (DC): -},{},1,1,3,3
3,PCG,Business interruption from labor dispute,{การหยุดชะงักของธุรกิจจากข้อพิพาทแรงงาน ครอบคล...,Operational Risk,"{Lack of Business Continuity Planning: -, Lack...",{},"{Legal: -, Human Resources: -, Corporate Commu...",{},1,1,1,1
4,PCG,Business interruption from natural disasters,{เกิดพายุลมแรง พัดผ่านอาคารคลังสินค้า ส่งผลให้...,Operational Risk,"{Lack of Emergency Response Training: -, Outda...",{},"{Engineering: -, Distribution Center (DC): -}",{},2,2,3,6


In [10]:
df2.company.value_counts()

company
PCG    66
Name: count, dtype: int64

In [11]:
df3 = df2[
    df2["company"].isin(
        [
            "PCG",
        ]
    )
]
print(df3.shape)
df3.company.value_counts()

(66, 12)


company
PCG    66
Name: count, dtype: int64

In [12]:
df3.head()

,company,risk,risk_desc,risk_cat,rootcause,rootcause_desc,process,process_desc,risk_level,likelihood_combined,impact_combined,risk_score
0,PCG,Accounting errors,{การแจ้งรายละเอียดค่าใช้จ่ายที่เกิดขึ้นในคลังม...,Operational Risk,"{Mistakes in calculations, data entry, or judg...",{},{Finance and Accounting: -},{},1,2,1,2
1,PCG,Business interruption from fire hazards,{กระบวนการทำงานของบริษัทมีความเสี่ยงที่อาจก่อใ...,Operational Risk,"{Inadequate Fire Safety Training: -, Insuffici...",{},"{Maintenance: -, Engineering: -, Production: -...",{},2,2,3,6
2,PCG,Business interruption from flood,{น้ำท่วมพื้นที่คลังสินค้า หรือบริเวณใกล้เคียง},Operational Risk,"{Inadequate Emergency Training: -, Poor Commun...",{},{Distribution Center (DC): -},{},1,1,3,3
3,PCG,Business interruption from labor dispute,{การหยุดชะงักของธุรกิจจากข้อพิพาทแรงงาน ครอบคล...,Operational Risk,"{Lack of Business Continuity Planning: -, Lack...",{},"{Legal: -, Human Resources: -, Corporate Commu...",{},1,1,1,1
4,PCG,Business interruption from natural disasters,{เกิดพายุลมแรง พัดผ่านอาคารคลังสินค้า ส่งผลให้...,Operational Risk,"{Lack of Emergency Response Training: -, Outda...",{},"{Engineering: -, Distribution Center (DC): -}",{},2,2,3,6


In [13]:
df3.columns

Index(['company', 'risk', 'risk_desc', 'risk_cat', 'rootcause',
       'rootcause_desc', 'process', 'process_desc', 'risk_level',
       'likelihood_combined', 'impact_combined', 'risk_score'],
      dtype='object')

In [14]:
select_columns = [
    "company",
    "risk_cat",
    "risk",
    "risk_desc",
    "rootcause",
    "rootcause_desc",
    "process",
    "process_desc",
    "risk_level",
]
df4 = df3[select_columns]
print(df4.shape)
df4.head()

(66, 9)


,company,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
0,PCG,Operational Risk,Accounting errors,{การแจ้งรายละเอียดค่าใช้จ่ายที่เกิดขึ้นในคลังม...,"{Mistakes in calculations, data entry, or judg...",{},{Finance and Accounting: -},{},1
1,PCG,Operational Risk,Business interruption from fire hazards,{กระบวนการทำงานของบริษัทมีความเสี่ยงที่อาจก่อใ...,"{Inadequate Fire Safety Training: -, Insuffici...",{},"{Maintenance: -, Engineering: -, Production: -...",{},2
2,PCG,Operational Risk,Business interruption from flood,{น้ำท่วมพื้นที่คลังสินค้า หรือบริเวณใกล้เคียง},"{Inadequate Emergency Training: -, Poor Commun...",{},{Distribution Center (DC): -},{},1
3,PCG,Operational Risk,Business interruption from labor dispute,{การหยุดชะงักของธุรกิจจากข้อพิพาทแรงงาน ครอบคล...,"{Lack of Business Continuity Planning: -, Lack...",{},"{Legal: -, Human Resources: -, Corporate Commu...",{},1
4,PCG,Operational Risk,Business interruption from natural disasters,{เกิดพายุลมแรง พัดผ่านอาคารคลังสินค้า ส่งผลให้...,"{Lack of Emergency Response Training: -, Outda...",{},"{Engineering: -, Distribution Center (DC): -}",{},2


In [15]:
# select first row of column risk_desc
cell = df4.iloc[4, 2]
# cell = df4.iloc[2, 3]
# cell = df4.iloc[3, 3]
# print(type(cell))
# print(cell)
import json
import traceback
from typing import Union


def list_string_to_list(list_string) -> tuple[bool, Union[list, str]]:
    is_list = False
    if ((list_string.count("'") % 2 != 0) and (list_string.count("'") != 0)) or (
        (list_string.count('"') % 2 != 0) and (list_string.count('"') != 0)
    ):

        return is_list, list_string
    # print(f"pass list_string :{type(list_string)} {list_string}")
    is_list = True
    # swap "'" to '"' and swap '"' to "'"
    list_string = list_string.replace('"', "###")
    list_string = list_string.replace("'", '"')
    list_string = list_string.replace("###", "'")
    try:
        list_cell = json.loads(list_string)
    except:
        is_list = False
        # print(f"here list_string : {list_string}")
        # print(traceback.format_exc())
        return is_list, list_string
    return is_list, list_cell


def cell_to_list(cell):
    empty_element_list = ["", "-"]
    if type(cell) == set:
        new_cell = []
        for list_string_candidate in cell:
            if list_string_candidate in empty_element_list:
                continue
            try:
                # if list_string count "'" or '"' have to be even and not 0
                # print(f"list_string_candidate : >>>{list_string_candidate}<<<")

                is_list, new_list_string_candidate = list_string_to_list(
                    list_string_candidate
                )
                # print(new_list_string_candidate == list_string_candidate)
                # print("is_list", is_list)
                # print(f"new_list_string_candidate : >>>{new_list_string_candidate}<<<")
                if is_list is False:
                    new_cell.append(new_list_string_candidate)
                    continue
                for i in new_list_string_candidate:
                    if i in empty_element_list:
                        continue

                    is_list, i = list_string_to_list(i)
                    # print(f"i : {is_list}>>>{i}<<<")
                    if is_list is False:
                        new_cell.append(i)
                        continue
                    new_cell.extend(i)

            except:
                # print(f"list_string : {list_string}")
                # print(traceback.format_exc())
                new_cell.append(list_string)
        return new_cell
    return cell


result = cell_to_list(cell)
print(cell_to_list(cell))
print(type(cell_to_list(cell)))

# print(cell_to_list(cell)[0])
# print(type(cell_to_list(cell)[0]))

Business interruption from natural disasters
<class 'str'>


In [16]:
result

'Business interruption from natural disasters'

In [17]:
list_string = "ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้นตอน ซึ่งขึ้นอยู่แต่ละปัญหาของสินค้า"
if ((list_string.count("'") % 2 != 0) and (list_string.count("'") != 0)) or (
    (list_string.count('"') % 2 != 0) and (list_string.count('"') != 0)
):
    print("yes")

In [18]:
# df5 is df4 with pass each cell with cell_to_list
# convert set of list_string to list
df5 = df4.applymap(cell_to_list)
df5.head()

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_75694/139688506.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df5 = df4.applymap(cell_to_list)


,company,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
0,PCG,Operational Risk,Accounting errors,[การแจ้งรายละเอียดค่าใช้จ่ายที่เกิดขึ้นในคลังม...,"[Mistakes in calculations, data entry, or judg...",[],[Finance and Accounting: -],[],1
1,PCG,Operational Risk,Business interruption from fire hazards,[กระบวนการทำงานของบริษัทมีความเสี่ยงที่อาจก่อใ...,"[Inadequate Fire Safety Training: -, Insuffici...",[],"[Maintenance: -, Engineering: -, Production: -...",[],2
2,PCG,Operational Risk,Business interruption from flood,[น้ำท่วมพื้นที่คลังสินค้า หรือบริเวณใกล้เคียง],"[Inadequate Emergency Training: -, Poor Commun...",[],[Distribution Center (DC): -],[],1
3,PCG,Operational Risk,Business interruption from labor dispute,[การหยุดชะงักของธุรกิจจากข้อพิพาทแรงงาน ครอบคล...,"[Lack of Business Continuity Planning: -, Lack...",[],"[Legal: -, Human Resources: -, Corporate Commu...",[],1
4,PCG,Operational Risk,Business interruption from natural disasters,[เกิดพายุลมแรง พัดผ่านอาคารคลังสินค้า ส่งผลให้...,"[Lack of Emergency Response Training: -, Outda...",[],"[Engineering: -, Distribution Center (DC): -]",[],2


In [19]:
# df6 convert list to string with join
df6 = df5.applymap(lambda x: ",".join(x) if isinstance(x, list) else x)
df6.head()

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_75694/3479902738.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df6 = df5.applymap(lambda x: ",".join(x) if isinstance(x, list) else x)


,company,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
0,PCG,Operational Risk,Accounting errors,การแจ้งรายละเอียดค่าใช้จ่ายที่เกิดขึ้นในคลังมี...,"Mistakes in calculations, data entry, or judgm...",,Finance and Accounting: -,,1
1,PCG,Operational Risk,Business interruption from fire hazards,กระบวนการทำงานของบริษัทมีความเสี่ยงที่อาจก่อให...,"Inadequate Fire Safety Training: -, Insufficie...",,"Maintenance: -, Engineering: -, Production: -,...",,2
2,PCG,Operational Risk,Business interruption from flood,น้ำท่วมพื้นที่คลังสินค้า หรือบริเวณใกล้เคียง,"Inadequate Emergency Training: -, Poor Communi...",,Distribution Center (DC): -,,1
3,PCG,Operational Risk,Business interruption from labor dispute,การหยุดชะงักของธุรกิจจากข้อพิพาทแรงงาน ครอบคลุ...,"Lack of Business Continuity Planning: -, Lack ...",,"Legal: -, Human Resources: -, Corporate Commun...",,1
4,PCG,Operational Risk,Business interruption from natural disasters,เกิดพายุลมแรง พัดผ่านอาคารคลังสินค้า ส่งผลให้อ...,"Lack of Emergency Response Training: -, Outdat...",,"Engineering: -,Distribution Center (DC): -",,2


In [20]:
df7 = df6.copy()


# new_rootcause is 'rootcause : ' + rootcause column +\n+ 'rootcause_desc : ' rootcause_desc if it not '' empty string
# fundction to apply
def merge_rootcause(row):
    rootcause = row["rootcause"]
    rootcause_desc = row["rootcause_desc"]
    if rootcause == "" and rootcause_desc == "":
        return ""
    elif rootcause == "":
        return "rootcause_desc :" + rootcause_desc
    elif rootcause_desc == "":
        return "rootcause :" + rootcause
    else:
        return "rootcause :" + rootcause + "\n" + "rootcause_desc :" + rootcause_desc


df7["new_rootcause"] = df7.apply(merge_rootcause, axis=1)


def merge_process(row):
    process = row["process"]
    process_desc = row["process_desc"]
    if process == "" and process_desc == "":
        return ""
    elif process == "":
        return "process_desc :" + process_desc
    elif process_desc == "":
        return "process :" + process
    else:
        return "process :" + process + "\n" + "process_desc :" + process_desc


df7["new_process"] = df7.apply(merge_process, axis=1)
# remove rootcause and process column
df7.drop(
    ["rootcause", "rootcause_desc", "process", "process_desc"], axis=1, inplace=True
)
# rename new_rootcause and new_process to rootcause and process
df7.rename(
    columns={"new_rootcause": "rootcause", "new_process": "process"}, inplace=True
)
df7.head()

,company,risk_cat,risk,risk_desc,risk_level,rootcause,process
0,PCG,Operational Risk,Accounting errors,การแจ้งรายละเอียดค่าใช้จ่ายที่เกิดขึ้นในคลังมี...,1,"rootcause :Mistakes in calculations, data entr...",process :Finance and Accounting: -
1,PCG,Operational Risk,Business interruption from fire hazards,กระบวนการทำงานของบริษัทมีความเสี่ยงที่อาจก่อให...,2,"rootcause :Inadequate Fire Safety Training: -,...","process :Maintenance: -, Engineering: -, Produ..."
2,PCG,Operational Risk,Business interruption from flood,น้ำท่วมพื้นที่คลังสินค้า หรือบริเวณใกล้เคียง,1,"rootcause :Inadequate Emergency Training: -, P...",process :Distribution Center (DC): -
3,PCG,Operational Risk,Business interruption from labor dispute,การหยุดชะงักของธุรกิจจากข้อพิพาทแรงงาน ครอบคลุ...,1,rootcause :Lack of Business Continuity Plannin...,"process :Legal: -, Human Resources: -, Corpora..."
4,PCG,Operational Risk,Business interruption from natural disasters,เกิดพายุลมแรง พัดผ่านอาคารคลังสินค้า ส่งผลให้อ...,2,rootcause :Lack of Emergency Response Training...,"process :Engineering: -,Distribution Center (D..."


In [21]:
risk_df_path = "data/RMI-V2-Translate_20250508.xlsx"
risk_df = pd.read_excel(risk_df_path, sheet_name="Risks")
risk_cause_mapping_df = pd.read_excel(risk_df_path, sheet_name="Risk_Cause_mapping")

selected_cols = [
    "Risk-EN",
    "Description-EN",
]

# risk_df["Risk-category"]# replace Operational risk with Operational Risk
risk_df["Risk-category"] = risk_df["Risk-category"].replace(
    "Operational risk", "Operational Risk"
)
for idx, row in risk_df.iterrows():
    risk_en = row["Risk-EN"]
    # risk_cause_mapping_df column RiskName get all RiskCause
    risk_cause_mapping_df_row = risk_cause_mapping_df[
        risk_cause_mapping_df["RiskName"] == risk_en
    ]
    # get all RiskCause
    risk_cause_list = risk_cause_mapping_df_row["RiskCause"].tolist()
    risk_cause_str = "\n".join(["- " + i for i in risk_cause_list])
    # add risk_cause_str to column Root cause-EN
    risk_df.loc[idx, "Root cause-EN"] = risk_cause_str


print(risk_df.columns)
risk_and_risk_desc_df = risk_df[selected_cols]
# loop each row in risk_and_risk_desc_df:
# apply strip to all columns
risk_and_risk_desc_df = risk_and_risk_desc_df.applymap(lambda x: x.strip())

risk_and_risk_desc_df.head()
# create map_dict
risk_to_desc_dict = risk_and_risk_desc_df.set_index("Risk-EN")[
    "Description-EN"
].to_dict()
len(risk_to_desc_dict)

/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Index(['GroupingIndex', 'Grouping Question Name', 'Code', 'Risk-category',
       'Risk-Sub-Cat-EN', 'Risk-EN', 'Risk-TH', 'Risk-CN', 'Description-EN',
       'Description-TH', 'Description-CN', 'Unnamed: 11', 'Root cause-EN'],
      dtype='object')


/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_75694/3047557379.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  risk_and_risk_desc_df = risk_and_risk_desc_df.applymap(lambda x: x.strip())


314

In [22]:
df8 = df7.copy()
# print number cell == NaN
print(df8.isnull().sum().sum())
# risk column map with risk_to_desc_dict
# apply strip to df8["risk"]
df8["risk"] = df8["risk"].str.strip()
df8["risk_desc_catalog"] = df8["risk"].map(risk_to_desc_dict)

df8.head()
# is nan in risk_desc_catalog
print(df8["risk_desc_catalog"].isnull().sum())
# fill risk_desc_catalog nan with ""
df8["risk_desc_catalog"] = df8["risk_desc_catalog"].fillna("")
df8["risk_desc_catalog"] = df8["risk_desc_catalog"].str.strip()
# print row that risk_desc_catalog is nan
df8[df8["risk_desc_catalog"].isnull()]
# concat risk_desc_catalog and risk_desc
df8["risk_desc"] = df8["risk_desc_catalog"] + " " + df8["risk_desc"]
# strip df8["risk_desc"]
df8["risk_desc"] = df8["risk_desc"].str.strip()
# remove risk_desc_catalog
df8.drop("risk_desc_catalog", axis=1, inplace=True)
all_risk_cat = risk_df["Risk-category"].unique().tolist()
# check is all df8['risk_cat'] in all_risk_cat
assert all([x in all_risk_cat for x in df8["risk_cat"].unique().tolist()])
# add new data to df8 from risk_and_risk_cat_and_risk_desc_df
print(df8.shape)

new_rows = []
for index, row in risk_df.iterrows():
    risk_cat = row["Risk-category"]
    if risk_cat not in selected_risk_cat:
        continue
    risk_desc = row["Description-EN"]
    risk_name = row["Risk-EN"]
    risk_rootcause = row["Root cause-EN"]
    data = {
        # company	risk_cat	risk	risk_desc	rootcause	process
        "company": f"risk_catalog-{date_stamp}",
        "risk_cat": risk_cat,
        "risk": risk_name,
        "risk_desc": risk_desc,
        "rootcause": risk_rootcause,
        "process": "",
        "risk_level": 0,
    }
    new_rows.append(data)
df8 = pd.concat([df8, pd.DataFrame(new_rows)], ignore_index=True)
print(df8.shape)
print("nan", df8.isnull().sum().sum())

df8.head()

0
0
(66, 7)
(300, 7)
nan 0


,company,risk_cat,risk,risk_desc,risk_level,rootcause,process
0,PCG,Operational Risk,Accounting errors,Accounting error refers to inaccuracies in the...,1,"rootcause :Mistakes in calculations, data entr...",process :Finance and Accounting: -
1,PCG,Operational Risk,Business interruption from fire hazards,The business interruption due to a fire incide...,2,"rootcause :Inadequate Fire Safety Training: -,...","process :Maintenance: -, Engineering: -, Produ..."
2,PCG,Operational Risk,Business interruption from flood,The business interruption due to flooding in t...,1,"rootcause :Inadequate Emergency Training: -, P...",process :Distribution Center (DC): -
3,PCG,Operational Risk,Business interruption from labor dispute,Business interruption from labor dispute encom...,1,rootcause :Lack of Business Continuity Plannin...,"process :Legal: -, Human Resources: -, Corpora..."
4,PCG,Operational Risk,Business interruption from natural disasters,Business interruption from natural disasters s...,2,rootcause :Lack of Emergency Response Training...,"process :Engineering: -,Distribution Center (D..."


In [23]:
all([x in all_risk_cat for x in df8["risk_cat"].unique().tolist()])

True

In [24]:
all_risk_cat

['Operational Risk',
 'Strategic Risk',
 'Market Risk',
 'Credit Risk',
 'Liquidity Risk',
 'A. Setting up business\n\n',
 'B. Business operation',
 'C. Compliance and regulator',
 'D. Policy',
 'E. Raising Concerns and Seeking Guidance',
 'F. Review, Assessment and Improvement']

In [25]:
df8["risk_cat"].unique().tolist()

['Operational Risk',
 'Credit Risk',
 'Market Risk',
 'Strategic Risk',
 'Liquidity Risk']

In [26]:
df.shape

(175, 27)

In [27]:
df8.shape

(300, 7)

In [28]:
df_final = df8.copy()

In [29]:
# convert df6:dataframe to data_dict
result_list = df_final.to_dict(orient="records")
result_list[:3]

[{'company': 'PCG',
  'risk_cat': 'Operational Risk',
  'risk': 'Accounting errors',
  'risk_desc': "Accounting error refers to inaccuracies in the recording, calculation, or reporting of financial transactions within an organization's accounting system. These errors can vary from simple mathematical mistakes to more complex issues like misapplication of accounting principles or estimation errors. การแจ้งรายละเอียดค่าใช้จ่ายที่เกิดขึ้นในคลังมีความคลาดเคลื่อน ส่งผลให้ค่าใช้จ่ายด้านโลจิสติกส์ (Logistic Cost) ไม่สะท้อนถึงความเป็นจริง และอาจทำให้การคำนวณค่าใช้จ่ายไม่สมเหตุสมผล",
  'risk_level': 1,
  'rootcause': 'rootcause :Mistakes in calculations, data entry, or judgment errors during the accounting process: -',
  'process': 'process :Finance and Accounting: -'},
 {'company': 'PCG',
  'risk_cat': 'Operational Risk',
  'risk': 'Business interruption from fire hazards',
  'risk_desc': 'The business interruption due to a fire incident that occurs within the workplace premises can result in 

In [30]:
# save it as json
import os

dir_path = os.getcwd()
save_file_path = f"{dir_path}/result/250528-company_risk_data.json"
# save result_list as json
import json

with open(save_file_path, "w") as f:
    json.dump(result_list, f, ensure_ascii=False)